# MSM Calibrator

In this notebook we implement the function `calibrate_params_ABM` that will be used for calibrating the bounded confidence model in all the scenarios.

It is base on `blackit`implementation of the Calibrator.

In [1]:
import numpy as np
import sys
sys.path += ["../src"]
from calibrator_blackit import Calibrator #i just commented few lines of code to remove the output print
#from black_it.calibrator import Calibrator
from black_it.samplers.halton import HaltonSampler
from black_it.loss_functions.msm import MethodOfMomentsLoss
import pandas as pd
from time import time

In [2]:
def calibrate_params_ABM(simulator, parameters_names = ["epsilon"], calibration_batches = 100, ensemble_size = 3,
                         sampler = HaltonSampler, loss = MethodOfMomentsLoss(), batch_size = 1,
                         parameters_bounds = [[0.], [0.5]], precisions = [0.00001]):
    
    t0 = time()
    model = simulator.simulate_ts
    real_data = simulator.ts
    
    samplers = [sampler(batch_size)]


    BC_cal = Calibrator(real_data = real_data,
                        samplers = samplers,
                        loss_function = loss,
                        model = model,
                        parameters_bounds = parameters_bounds,
                        parameters_precision = precisions,
                        ensemble_size = ensemble_size,
                        saving_folder = None,
                        verbose = False,
                        n_jobs = 1#'initial_state'
                       )
    
    
    params, losses = BC_cal.calibrate(calibration_batches)
    t1 = time()
    
    output_dict = {"loss": losses}
    for i,param_i in enumerate(parameters_names):
        output_dict[param_i] = params.T[i]
        
    return pd.DataFrame(output_dict), t1 - t0

### Example

In [3]:
from MSM_simulators import FBCM_simulator, PBCM_simulator, NBCM_simulator
import simulator_opinion_dynamics as sod

In [4]:
N, T, edge_per_t = 100, 256, 4
evidences_per_t = 4
epsilon, mu, rho = 0.35, 0.4, 16

X, edges, evidences = sod.simulate_BC(N, T, edge_per_t, evidences_per_t, (epsilon, mu, rho), seed = 34945)

In [5]:
fbcm_simulator = FBCM_simulator(X[0], edges, N, mu, epsilon)

In [6]:
fbcm_calibration = calibrate_params_ABM(fbcm_simulator, parameters_names = ["epsilon"], calibration_batches = 100, ensemble_size = 3,
                                        sampler = HaltonSampler, loss = MethodOfMomentsLoss(), batch_size = 1,
                                        parameters_bounds = [[0.], [0.5]], precisions = [0.00001])
    

In [7]:
fbcm_calibration[0].head()

,loss,epsilon
0,0.047991,0.33463
1,0.074657,0.38931
2,0.166830,0.28287
3,0.171490,0.29849
4,0.194923,0.32974


In [8]:
epsilon_estimate = fbcm_calibration[0]["epsilon"][0]
print(f"Correct epsilon {epsilon}")
print(f"Estimated epsilon {round(epsilon_estimate, 3)}")
print(f"Number of simulations {len(fbcm_calibration[0])}")
print(f"Estimation time {round(fbcm_calibration[1], 1)}s")

Correct epsilon 0.35
Estimated epsilon 0.335
Number of simulations 100
Estimation time 29.0s
